In [1]:
# Import all the dependencies
from qdrant_client import models, QdrantClient
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain.vectorstores.qdrant import Qdrant
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
from peft import PeftModel, PeftConfig
#
from tqdm.auto import tqdm
from uuid import uuid4
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from sentence_transformers import SentenceTransformer, util
import pandas as pd

#
import os
import random
import torch

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the document that you need to parse, please change the location to where the pdf resides

# Load 1 PDF file


# loader = PyPDFDirectoryLoader("/mnt/data/" + os.environ['DOMINO_PROJECT_NAME'] + "/RAG/" + pdf_name)

# or load an entire folder
loader = PyPDFDirectoryLoader("/mnt/data/" + os.environ['DOMINO_PROJECT_NAME'] + "/RAG_Docs/")
data = loader.load_and_split(RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0))

In [3]:
print(f"There are {len(data)} pages in the document")

There are 1116 pages in the document


In [4]:
# Pick a sample page
print(data[random.randint(0, len(data))])

page_content='2. Select an option.\nHalf-press\nSpecify the function performed by pressing the shutter button halfway.\nFully-press\nSpecify the function performed by pressing the shutter button\ncompletely.\nWith [Fully-press] set to [ Start/stop mov rec], you can start/stop movie\nrecording not only with the movie shooting button but also by pressing the\nshutter button completely, or by using Remote Switch TC-80N3  (sold\nseparately).\n357' metadata={'source': '/mnt/data/LocalRAG-JW/RAG_Docs/Canon EOS R5 Advanced User Guide.pdf', 'page': 356}


In [5]:
# Split the data into pages
metadatas = []
texts = []
for row in data:
  metadatas.append(row.metadata)
  texts.append(row.page_content)
print(len(metadatas),len(texts))

1116 1116


In [6]:
# Setup the prompt template to use for the QA bot
prompt_template = """Use the following pieces of context to answer the question enclosed within  3 backticks at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Please provide an answer which is factually correct and based on the information retrieved from the vector store.
Please also mention any quotes supporting the answer if any present in the context supplied within two double quotes "" .

{context}

QUESTION:```{question}```
ANSWER:
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context","question"])
#


In [7]:
# Load the embedding model
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding_model_name = "BAAI/bge-small-en"
os.environ['SENTENCE_TRANSFORMERS_HOME'] = '/mnt/data/' + os.environ['DOMINO_PROJECT_NAME'] + '/model_cache/'
embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-small-en",
                                      model_kwargs=model_kwargs,
                                      encode_kwargs=encode_kwargs
                                     )

.gitattributes: 100%|██████████| 1.52k/1.52k [00:00<00:00, 218kB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 31.5kB/s]
README.md: 100%|██████████| 90.8k/90.8k [00:00<00:00, 63.0MB/s]
config_sentence_transformers.json: 100%|██████████| 124/124 [00:00<00:00, 71.9kB/s]
model.safetensors: 100%|██████████| 133M/133M [00:00<00:00, 291MB/s] 
pytorch_model.bin: 100%|██████████| 134M/134M [00:04<00:00, 30.6MB/s] 
sentence_bert_config.json: 100%|██████████| 52.0/52.0 [00:00<00:00, 7.80kB/s]
special_tokens_map.json: 100%|██████████| 125/125 [00:00<00:00, 57.2kB/s]
tokenizer.json: 100%|██████████| 711k/711k [00:00<00:00, 29.9MB/s]
tokenizer_config.json: 100%|██████████| 366/366 [00:00<00:00, 221kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 3.11MB/s]
modules.json: 100%|██████████| 349/349 [00:00<00:00, 206kB/s]


In [8]:
# Uncomment this code if you want to store the embeddings in Qdrant in-memory
doc_store = Qdrant.from_texts(texts,
                              metadatas=metadatas,
                              embedding=embeddings,
                              location=":memory:",
                              collection=f"{embedding_model_name}_arxiv_paper")

In [9]:
 # Persist the embeddings to disk
# doc_store = Qdrant.from_texts(texts,
#                               metadatas=metadatas,
#                               embedding=embeddings,
#                               path="/mnt/artifacts/local_qdrant/",
#                               prefer_grpc=True,
#                               collection=f"{embedding_model_name}_arxiv_paper")

In [10]:
# Load the model and the tokenizer
chain_type_kwargs = {"prompt": PROMPT}


################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

model_id = "NousResearch/Llama-2-7b-chat-hf"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    cache_dir="/mnt/data/" + os.environ['DOMINO_PROJECT_NAME'] + "/model_cache/",
    quantization_config=bnb_config,
    device_map='auto'
)

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.pad_token_id = model.config.eos_token_id

config.json: 100%|██████████| 583/583 [00:00<00:00, 337kB/s]
model.safetensors.index.json: 100%|██████████| 26.8k/26.8k [00:00<00:00, 15.6MB/s]
model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]
model-00001-of-00002.safetensors:   0%|          | 10.5M/9.98G [00:00<03:45, 44.1MB/s]
model-00001-of-00002.safetensors:   0%|          | 41.9M/9.98G [00:00<01:09, 142MB/s] 
model-00001-of-00002.safetensors:   1%|          | 83.9M/9.98G [00:00<00:45, 217MB/s]
model-00001-of-00002.safetensors:   1%|▏         | 126M/9.98G [00:00<00:36, 266MB/s] 
model-00001-of-00002.safetensors:   2%|▏         | 168M/9.98G [00:00<00:31, 308MB/s]
model-00001-of-00002.safetensors:   2%|▏         | 220M/9.98G [00:00<00:27, 361MB/s]
model-00001-of-00002.safetensors:   3%|▎         | 262M/9.98G [00:00<00:25, 376MB/s]
model-00001-of-00002.safetensors:   3%|▎         | 304M/9.98G [00:01<00:25, 376MB/s]
model-00001-of-00002.safetensors:   4%|▎         | 357M/9.98G [00:01<00:24, 396MB/s]
model-0

In [11]:
# Setup the QA chain
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100)
rag_llm = HuggingFacePipeline(pipeline=pipe)
    
qa_chain = RetrievalQA.from_chain_type(llm=rag_llm,
                                       chain_type="stuff",
                                       chain_type_kwargs={"prompt": PROMPT},
                                       retriever=doc_store.as_retriever(search_kwargs={"k": 5}),
                                       return_source_documents=True
                                      )

In [12]:
# Ask a question
user_question = input("Please provide your question here :")
result = qa_chain(user_question)
result['result']

# What the role of gravitational and drag forces on the concentration of large dust grains?

Please provide your question here : how do i use the interval timer?


/usr/local/lib/python3.8/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.8/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


'The interval timer is a feature that allows you to set the camera to take a series of photos at regular intervals. To use the interval timer, follow these steps:\n\n1. Select [ : Interval timer].\n2. Select [Enable].\nSelect [Enable], then press the < > button.\n3. Set the shooting interval and number of shots.\nSelect an option to set (hours : minutes : seconds / number of shots).\nPress < > to'